*This is fakeRoBERTa*

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.1 MB/s 
     |████████████████████████████████| 182 kB 75.4 MB/s 
     |████████████████████████████████| 7.6 MB 57.0 MB/s 


In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from sklearn.model_selection import train_test_split

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
encoded_label_dict = {"CG" : 0, "OR" : 1}
def encode_label(x):
    return encoded_label_dict.get(x,-1)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls /content/drive/MyDrive/YelpDataset/generatedData/GPT3

Generated_Sentences_RestaurantGPT3.csv
Generated_Sentences_RestaurantGPT3UnProcessed.csv


In [7]:
%cp /content/drive/MyDrive/YelpDataset/generatedData/GPT3/Generated_Sentences_RestaurantGPT3.csv /content/

In [8]:
df = pd.read_csv("Generated_Sentences_RestaurantGPT3.csv", encoding='latin-1')

In [9]:
df.head()

,category,rating,label,text_
0,Restaurants,4.0,CG,Excellent food. Large servings. Great burger...
1,Restaurants,4.0,OR,best chicken with green and yellow sauce cilan...
2,Restaurants,4.0,CG,"Hit the spot. Great food, friendly servers. Pr..."
3,Restaurants,4.5,OR,"Consider yourself lucky if you can get one, bo..."
4,Restaurants,3.5,CG,"Well, I thought it was awesome. The restaurant..."


In [10]:
df.shape

(1042, 4)

In [11]:
df.dropna(inplace=True)

In [12]:
df.shape

(1042, 4)

In [13]:
df["target"] = df["label"].apply(lambda x: encode_label(x))

In [14]:
df["category"].unique()

array(['Restaurants'], dtype=object)

In [15]:
df.describe()

,rating,target
count,1042.000000,1042.00000
mean,3.847889,0.50000
std,0.618045,0.50024
min,1.500000,0.00000
25%,3.500000,0.00000
50%,4.000000,0.50000
75%,4.500000,1.00000
max,5.000000,1.00000


In [16]:

MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 1
LEARNING_RATE = 1e-05

In [17]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.text_[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.target[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [19]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch

In [20]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [21]:
# Creating the dataset and dataloader
train_dataset, valid_dataset = train_test_split(df, test_size=0.2, shuffle=True, stratify=None, random_state=2021)
train_dataset = train_dataset.reset_index(drop=True)
valid_dataset = valid_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALID Dataset: {}".format(valid_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(valid_dataset, tokenizer, MAX_LEN)

FULL Dataset: (1042, 5)
TRAIN Dataset: (833, 5)
VALID Dataset: (209, 5)


#Without Finetune on the New Data

In [22]:
#/content/drive/MyDrive/YelpDataset/Roberta replace it with whereever you have the finetuned model saved in your drive
%cp -r /content/drive/MyDrive/YelpDataset/Roberta /content/

In [23]:
output_model_file = '/content/Roberta/ft-roberta-yelpreviews_kitchen.pt'

In [24]:
# model = torch.load('../../data/classification/models/ft-roberta-amazonreviews.pt')
model = torch.load(output_model_file)

In [25]:
def predict(query, model, tokenizer, device="cuda"):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.model_max_length - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
        logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
        probs = logits.softmax(dim=-1)

    fake, real = probs.detach().cpu().flatten().numpy().tolist()
    return real

In [26]:
query = """I work in the wedding industry and have to work long days, on my feet, outside in the heat, and have to look professional. I've spent a ridiculous amount of money on high end dress shoes like Merrels and just have not been able to find a pair that are comfortable to wear all day. Both for my feet and my back. Enter the Sanuk yoga sling!!! These shoes are amazingly comfortable. Though, I will admit it took a few wears to get used to the feel of the yoga matte bottom. At first, it felt a little "sticky" to me, and the fabric part that goes through the toe area was a little thick and took some getting used to. I wore them for a few days before taking them out on a job and I can't get over how comfortable they are. Ii have been wearing these shoes now for 3 months, every work day and I am THRILLED. No more back pain, no more sore feet. I also wear these sometimes during my off time,mans every time I wear them, I get compliments on how cute and comfortable they look. The great thing about these shoes is the yoga matte bottom. It helps your feet grip to the shoe a bit, so your foot can just walk normally, without having to grip the shoe. You may not realize it, but with a lot of Sandals, your foot is having to work to keep the shoe on, changing the way you walk and stand and ultimately causing foot and back pain. Not with these! Also, the soft linen sits comfortably on your skin and breathes nicely in the heat. The only downside is the funky tan lines, which is why I am sure to alternate shoes on my days off, especially if I plan to be outside for most of the day. If it were not for that, I think these might be the only shoes I'd wear all summer. If you are looking for a reasonable priced, comfortable shoe that you can wear and walk in all day."""
predict(query,model,tokenizer)

0.999467670917511

In [27]:
preds, preds_probas = [],[]
for i, row in valid_dataset.iterrows():
    query = row["text_"]
    pred = predict(query,model,tokenizer)
    preds_probas.append(pred)
    if pred >= 0.5:
        preds.append(1)
    else:
        preds.append(0)

In [28]:
from sklearn.metrics import confusion_matrix
y_true = valid_dataset.target.values
y_pred = preds
confusion_matrix(y_true,y_pred)

array([[96, 10],
       [ 5, 98]])

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
acc = accuracy_score(y_true,y_pred)
precision = precision_score(y_true,y_pred)
recall = recall_score(y_true,y_pred)

In [30]:
print(f"Accuracy: {acc*100}; Precision:{precision*100}; Recall:{recall*100}")

Accuracy: 92.82296650717703; Precision:90.74074074074075; Recall:95.14563106796116


In [31]:
print(classification_report(y_true, y_pred, target_names=["CG","OR"]))

              precision    recall  f1-score   support

          CG       0.95      0.91      0.93       106
          OR       0.91      0.95      0.93       103

    accuracy                           0.93       209
   macro avg       0.93      0.93      0.93       209
weighted avg       0.93      0.93      0.93       209



#### Writing predictions to disc

In [32]:
preds_df_rows = []
for i, row in valid_dataset.iterrows():
    query = row["text_"]
    pred_prob = preds_probas[i]
    pred_label = preds[i]
    preds_df_rows.append([pred_prob,pred_label])
preds_df = pd.DataFrame(preds_df_rows, columns=["Finetune_Roberta_Model_Probability","Finetune_Roberta_Model_Prediction"])

In [33]:
preds_df.to_csv("roberta_predictionsGPT3WithoutFineTune2.csv", index=None)

In [34]:
%cp /content/roberta_predictionsGPT3WithoutFineTune2.csv /content/drive/MyDrive/YelpDataset/Roberta

#Finetune on New Data

In [35]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

valid_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **valid_params)

In [36]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch

In [37]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)

In [38]:
output_model_file = '/content/Roberta/ft-roberta-yelpreviews_kitchen.pt'

In [39]:
# model = torch.load('../../data/classification/models/ft-roberta-amazonreviews.pt')
model = torch.load(output_model_file)

In [40]:
#model = RobertaForSequenceClassification.from_pretrained(model_name)
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [41]:
# Creating the optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [42]:
# Function to calcuate the accuracy of the model
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [43]:
# Defining the training function on the 80% of the dataset for tuning the roberta model
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        
        optimizer.zero_grad()
        outputs = model(ids, attention_mask=mask, labels=targets)
        loss = outputs.loss
        logits = outputs.logits
        tr_loss += loss
        big_val, big_idx = torch.max(logits, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _!=0 and _%100==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 100 steps: {loss_step}")
            print(f"Training Accuracy per 100 steps: {accu_step}")

        loss.backward()
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [44]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    n_wrong = 0
    total = 0
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, attention_mask=mask, labels=targets)
            loss = outputs.loss
            logits = outputs.logits
            tr_loss += loss
            big_val, big_idx = torch.max(logits, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _!=0 and _%100==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu


In [45]:
tokenizer.pad_token_id

1

In [46]:
for epoch in range(EPOCHS):
    train(epoch)
# 4000 -> 10min
# 40,000 -> 100min 
# start from 17:54 -> 18:10

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 100 steps: 0.19425369799137115
Training Accuracy per 100 steps: 93.31683168316832
The Total Accuracy for Epoch 0: 93.51740696278512
Training Loss Epoch: 0.1890932023525238
Training Accuracy Epoch: 93.51740696278512


In [47]:
acc = valid(model, testing_loader)
print("Accuracy on validation data = %0.2f%%" % acc)

Validation Loss Epoch: 0.12739889323711395
Validation Accuracy Epoch: 95.69377990430623
Accuracy on validation data = 95.69%


In [48]:
# Save the model
# output_model_file = '../../data/classification/models/ft-roberta-amazonreviews.pt'
output_model_file = 'ft-roberta-yelpreviews_kitchenGPT3Finetuned2.pt'

model_to_save = model
torch.save(model_to_save, output_model_file)

print('All files saved')

All files saved


In [49]:
%cp /content/ft-roberta-yelpreviews_kitchenGPT3Finetuned2.pt /content/drive/MyDrive/YelpDataset/Roberta

#### Inference

In [ ]:
"""
#use this if you need to mount your drive
from google.colab import drive
drive.mount('/content/drive')
"""

In [50]:
#/content/drive/MyDrive/YelpDataset/Roberta replace it with whereever you have the finetuned model saved in your drive
!ls /content/drive/MyDrive/YelpDataset/Roberta

ft-roberta-yelpreviews_kitchenGPT3Finetuned2.pt
ft-roberta-yelpreviews_kitchenGPT3Finetuned.pt
ft-roberta-yelpreviews_kitchen.pt
roberta_predictions.csv
roberta_predictionsGPT3FineTune.csv
roberta_predictionsGPT3WithoutFineTune2.csv
roberta_predictionsGPT3WithoutFineTune.csv


In [51]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch

In [52]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)

In [53]:
output_model_file = 'ft-roberta-yelpreviews_kitchenGPT3Finetuned2.pt'

In [54]:
# model = torch.load('../../data/classification/models/ft-roberta-amazonreviews.pt')
model = torch.load(output_model_file)

In [55]:
device="cuda"
query = """I work in the wedding industry and have to work long days, on my feet, outside in the heat, and have to look professional. I've spent a ridiculous amount of money on high end dress shoes like Merrels and just have not been able to find a pair that are comfortable to wear all day. Both for my feet and my back. Enter the Sanuk yoga sling!!! These shoes are amazingly comfortable. Though, I will admit it took a few wears to get used to the feel of the yoga matte bottom. At first, it felt a little "sticky" to me, and the fabric part that goes through the toe area was a little thick and took some getting used to. I wore them for a few days before taking them out on a job and I can't get over how comfortable they are. Ii have been wearing these shoes now for 3 months, every work day and I am THRILLED. No more back pain, no more sore feet. I also wear these sometimes during my off time,mans every time I wear them, I get compliments on how cute and comfortable they look. The great thing about these shoes is the yoga matte bottom. It helps your feet grip to the shoe a bit, so your foot can just walk normally, without having to grip the shoe. You may not realize it, but with a lot of Sandals, your foot is having to work to keep the shoe on, changing the way you walk and stand and ultimately causing foot and back pain. Not with these! Also, the soft linen sits comfortably on your skin and breathes nicely in the heat. The only downside is the funky tan lines, which is why I am sure to alternate shoes on my days off, especially if I plan to be outside for most of the day. If it were not for that, I think these might be the only shoes I'd wear all summer. If you are looking for a reasonable priced, comfortable shoe that you can wear and walk in all day."""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens)
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.997197151184082
Fake Probability: 0.0028028751257807016


In [56]:
#is fake
device="cuda"
query = """terrible servile. not worthy of a second visit.We decided to try this place for the"""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.026194270700216293
Fake Probability: 0.9738057255744934


In [57]:
#is original from the dataset
device="cuda"
query = """Wonderfully prepared and beautifully presented food. Staff is very friendly. Bright and clean."""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.29891064763069153
Fake Probability: 0.7010893225669861


In [58]:
#is original from the dataset
device="cuda"
query = """Consistently good standard American and Mexican breakfast fare. The coffee not so impressive."""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.6281370520591736
Fake Probability: 0.3718630075454712


In [59]:
#I wrote it myself
device="cuda"
query = """I've been going here for a 2 years but in the last few months the quality of their food has become really bad. The portion size has reduced and their food doesn't taste as good as it used to."""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.36147746443748474
Fake Probability: 0.6385225653648376


In [60]:
#Is original
device="cuda"
query = """Good pizza, fair amount of toppings but they came kind of askew. The subs had a very generous amount of thinly sliced n breaded chicken on them. Decent flavor overall."""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.9610210657119751
Fake Probability: 0.0389789454638958


In [61]:
#Is original
device="cuda"
query = """The decore, the tables, the hostess, the cultural ambiance... all perfect!! But the food was not what I expected. Besides THE BEST Thai tea in town, I've had better Thai food. But overall a good date night location and a good experience."""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.9932860732078552
Fake Probability: 0.006713952403515577


In [62]:
#Is fake generated by ChatGPT
device="cuda"
query = """
The food at this restaurant is absolutely incredible. From the moment we walked in, we were greeted by the friendly staff and the mouthwatering smells coming from the kitchen. The service was top-notch and the presentation of the dishes was stunning. I would highly recommend this restaurant to anyone looking for a high-quality dining experience
"""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.1857033371925354
Fake Probability: 0.8142966628074646


In [63]:
#Is fake generated by ChatGPT
device="cuda"
query = """
This restaurant is a hidden gem. The food is always fresh and full of flavor, and the portions are generous. The staff are attentive and friendly, and they always make sure we have everything we need. I would highly recommend this restaurant to anyone looking for a delicious meal at a reasonable price.
"""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.016348792240023613
Fake Probability: 0.9836512207984924


In [64]:
#Is fake generated by ChatGPT
device="cuda"
query = """
Terrible service, would not recommend."""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.3199218213558197
Fake Probability: 0.6800782084465027


In [65]:

#Is from the dataset
device="cuda"
query = """
best chicken with green and yellow sauce cilantro rice.
"""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.46642154455184937
Fake Probability: 0.5335784554481506


In [66]:

#I wrote it
device="cuda"
query = """
This place has the best sushi I've had in a while. What brought me to this place was the unlimited menu- definitely worth every buck!
"""
tokens = tokenizer.encode(query,return_tensors="pt")
all_tokens = len(tokens[0])
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}")

Real Probability: 0.9799696803092957
Fake Probability: 0.02003030851483345


In [67]:
def predict(query, model, tokenizer, device="cuda"):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.model_max_length - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
        logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
        probs = logits.softmax(dim=-1)

    fake, real = probs.detach().cpu().flatten().numpy().tolist()
    return real

In [68]:
query = """I work in the wedding industry and have to work long days, on my feet, outside in the heat, and have to look professional. I've spent a ridiculous amount of money on high end dress shoes like Merrels and just have not been able to find a pair that are comfortable to wear all day. Both for my feet and my back. Enter the Sanuk yoga sling!!! These shoes are amazingly comfortable. Though, I will admit it took a few wears to get used to the feel of the yoga matte bottom. At first, it felt a little "sticky" to me, and the fabric part that goes through the toe area was a little thick and took some getting used to. I wore them for a few days before taking them out on a job and I can't get over how comfortable they are. Ii have been wearing these shoes now for 3 months, every work day and I am THRILLED. No more back pain, no more sore feet. I also wear these sometimes during my off time,mans every time I wear them, I get compliments on how cute and comfortable they look. The great thing about these shoes is the yoga matte bottom. It helps your feet grip to the shoe a bit, so your foot can just walk normally, without having to grip the shoe. You may not realize it, but with a lot of Sandals, your foot is having to work to keep the shoe on, changing the way you walk and stand and ultimately causing foot and back pain. Not with these! Also, the soft linen sits comfortably on your skin and breathes nicely in the heat. The only downside is the funky tan lines, which is why I am sure to alternate shoes on my days off, especially if I plan to be outside for most of the day. If it were not for that, I think these might be the only shoes I'd wear all summer. If you are looking for a reasonable priced, comfortable shoe that you can wear and walk in all day."""
predict(query,model,tokenizer)

0.9970579147338867

In [69]:
preds, preds_probas = [],[]
for i, row in valid_dataset.iterrows():
    query = row["text_"]
    pred = predict(query,model,tokenizer)
    preds_probas.append(pred)
    if pred >= 0.5:
        preds.append(1)
    else:
        preds.append(0)

In [70]:
from sklearn.metrics import confusion_matrix
y_true = valid_dataset.target.values
y_pred = preds
confusion_matrix(y_true,y_pred)

array([[103,   3],
       [  8,  95]])

In [71]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
acc = accuracy_score(y_true,y_pred)
precision = precision_score(y_true,y_pred)
recall = recall_score(y_true,y_pred)

In [72]:
print(f"Accuracy: {acc*100}; Precision:{precision*100}; Recall:{recall*100}")

Accuracy: 94.73684210526315; Precision:96.93877551020408; Recall:92.23300970873787


In [73]:
print(classification_report(y_true, y_pred, target_names=["CG","OR"]))

              precision    recall  f1-score   support

          CG       0.93      0.97      0.95       106
          OR       0.97      0.92      0.95       103

    accuracy                           0.95       209
   macro avg       0.95      0.95      0.95       209
weighted avg       0.95      0.95      0.95       209



#### Writing predictions to disc

In [74]:
preds_df_rows = []
for i, row in valid_dataset.iterrows():
    query = row["text_"]
    pred_prob = preds_probas[i]
    pred_label = preds[i]
    preds_df_rows.append([pred_prob,pred_label])
preds_df = pd.DataFrame(preds_df_rows, columns=["Finetune_Roberta_Model_Probability","Finetune_Roberta_Model_Prediction"])

In [75]:
preds_df.to_csv("roberta_predictionsGPT3FineTune2.csv", index=None)

In [76]:
%cp /content/roberta_predictionsGPT3FineTune2.csv /content/drive/MyDrive/YelpDataset/Roberta

# CHATGPT GENERATED DATA

In [94]:
valid_dataset = pd.read_csv("/content/ChatGPTGeneratedFakeReviews.csv")

In [95]:
valid_dataset.shape

(56, 2)

In [96]:
output_model_file = '/content/ft-roberta-yelpreviews_kitchenGPT3Finetuned2.pt'

In [97]:
# model = torch.load('../../data/classification/models/ft-roberta-amazonreviews.pt')
model = torch.load(output_model_file)

In [98]:
def predict(query, model, tokenizer, device="cuda"):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.model_max_length - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
        logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
        probs = logits.softmax(dim=-1)

    fake, real = probs.detach().cpu().flatten().numpy().tolist()
    return real

In [99]:
query = """I work in the wedding industry and have to work long days, on my feet, outside in the heat, and have to look professional. I've spent a ridiculous amount of money on high end dress shoes like Merrels and just have not been able to find a pair that are comfortable to wear all day. Both for my feet and my back. Enter the Sanuk yoga sling!!! These shoes are amazingly comfortable. Though, I will admit it took a few wears to get used to the feel of the yoga matte bottom. At first, it felt a little "sticky" to me, and the fabric part that goes through the toe area was a little thick and took some getting used to. I wore them for a few days before taking them out on a job and I can't get over how comfortable they are. Ii have been wearing these shoes now for 3 months, every work day and I am THRILLED. No more back pain, no more sore feet. I also wear these sometimes during my off time,mans every time I wear them, I get compliments on how cute and comfortable they look. The great thing about these shoes is the yoga matte bottom. It helps your feet grip to the shoe a bit, so your foot can just walk normally, without having to grip the shoe. You may not realize it, but with a lot of Sandals, your foot is having to work to keep the shoe on, changing the way you walk and stand and ultimately causing foot and back pain. Not with these! Also, the soft linen sits comfortably on your skin and breathes nicely in the heat. The only downside is the funky tan lines, which is why I am sure to alternate shoes on my days off, especially if I plan to be outside for most of the day. If it were not for that, I think these might be the only shoes I'd wear all summer. If you are looking for a reasonable priced, comfortable shoe that you can wear and walk in all day."""
predict(query,model,tokenizer)

0.9970579147338867

In [100]:
preds, preds_probas = [],[]
for i, row in valid_dataset.iterrows():
    query = row["text"]
    pred = predict(query,model,tokenizer)
    preds_probas.append(pred)
    if pred >= 0.5:
        preds.append(1)
    else:
        preds.append(0)

In [101]:
from sklearn.metrics import confusion_matrix
y_true = valid_dataset.label.values
y_pred = preds
confusion_matrix(y_true,y_pred)

array([[50,  3],
       [ 0,  3]])

In [102]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
acc = accuracy_score(y_true,y_pred)
precision = precision_score(y_true,y_pred)
recall = recall_score(y_true,y_pred)

In [103]:
print(f"Accuracy: {acc*100}; Precision:{precision*100}; Recall:{recall*100}")

Accuracy: 94.64285714285714; Precision:50.0; Recall:100.0


In [104]:
print(classification_report(y_true, y_pred, target_names=["CG","OR"]))

              precision    recall  f1-score   support

          CG       1.00      0.94      0.97        53
          OR       0.50      1.00      0.67         3

    accuracy                           0.95        56
   macro avg       0.75      0.97      0.82        56
weighted avg       0.97      0.95      0.95        56



In [105]:
%cp /content/ChatGPTGeneratedFakeReviews.csv /content/drive/MyDrive/YelpDataset/Roberta